In [13]:
import os
from datetime import datetime
import pydoop.hdfs as hdfs
import re

In [33]:
dirs = os.listdir('/home/itversity/itversity-material/retail_pipeline/data')
groups = [d for d in dirs if os.path.isdir(os.path.join('/home/itversity/itversity-material/retail_pipeline/data', d)) and d.startswith('group')]
print(groups)

['group', 'group1', 'group2', 'group3', 'group3 - copy', 'group4']


- Function to write ingested files into a text file to skip them on the next round

In [34]:
def write_dirs_to_file(file_path, group):
    with open(file_path, 'r') as file:
            existing_content = file.read().splitlines()
            
    with open(file_path, 'a') as file:
        if group not in existing_content:
            file.write(f"{group}\n")

In [35]:
with open('/home/itversity/itversity-material/retail_pipeline/data/written_groups.txt', 'r') as file:
    skip_dir = file.read().splitlines()
    
filtered_dir = [group for group in groups if group not in skip_dir]
if filtered_dir:
    files = os.listdir(f"/home/itversity/itversity-material/retail_pipeline/data/{filtered_dir[0]}")
else:
    print("No new directories to process.")
    
print(filtered_dir[0])

group1


 Function to rename the files

In [36]:
def rename_files_and_upload(local_directory, hdfs_directory):
    # Rename files
    for file_name in files:
        old_file_path = os.path.join(local_directory, file_name)
        
        # Get the current date and hour
        now = datetime.now()
        date_str = now.strftime("%Y%m%d")
        hour_str = now.strftime("%H")
        
        # Split the file name and extension
        split_name= os.path.splitext(file_name)
        print(split_name)
        # Create the new file name with the extension at the end
        new_file_name = f"{split_name[0][:-2]}_{date_str}_{hour_str}{split_name[1]}"
        new_file_path = os.path.join(local_directory, new_file_name)
        
        # Rename the file
        try:
            os.rename(old_file_path, new_file_path)
            print(f"Renamed {old_file_path} to {new_file_path}")
        except FileNotFoundError:
            print("no new files found")
        
        # Upload the renamed file to HDFS
        hdfs.mkdir(f"/data/retail_bronze/{date_str}/{hour_str}")
        day_directory = os.path.join(hdfs_directory, date_str)
        hour_directory = os.path.join(day_directory, hour_str)
        hdfs_file_path = os.path.join(hour_directory, new_file_name)
        hdfs.put(new_file_path, hdfs_file_path)
        print(f"Uploaded {new_file_path} to {hdfs_file_path}")

if filtered_dir:
    rename_files_and_upload(f"/home/itversity/itversity-material/retail_pipeline/data/{filtered_dir[0]}", '/data/retail_bronze')

('branches_SS_raw_1_20240630_12', '.csv')
Renamed /home/itversity/itversity-material/retail_pipeline/data/group1/branches_SS_raw_1_20240630_12.csv to /home/itversity/itversity-material/retail_pipeline/data/group1/branches_SS_raw_1_20240630__20240703_12.csv
Uploaded /home/itversity/itversity-material/retail_pipeline/data/group1/branches_SS_raw_1_20240630__20240703_12.csv to /data/retail_bronze/20240703/12/branches_SS_raw_1_20240630__20240703_12.csv
('sales_agents_SS_raw_1_20240630_12', '.csv')
Renamed /home/itversity/itversity-material/retail_pipeline/data/group1/sales_agents_SS_raw_1_20240630_12.csv to /home/itversity/itversity-material/retail_pipeline/data/group1/sales_agents_SS_raw_1_20240630__20240703_12.csv
Uploaded /home/itversity/itversity-material/retail_pipeline/data/group1/sales_agents_SS_raw_1_20240630__20240703_12.csv to /data/retail_bronze/20240703/12/sales_agents_SS_raw_1_20240630__20240703_12.csv
('sales_transactions_SS_raw_1_20240630_12', '.csv')
Renamed /home/itversity/

write the already ingested files to the log

In [37]:
write_dirs_to_file('/home/itversity/itversity-material/retail_pipeline/data/written_groups.txt', filtered_dir[0])